# Housing Data Wrangling

This script performs data cleaning and mapping operations on housing data for analysis. The dataset originates from the U.S. Census Bureau's Public Use Microdata Sample (PUMS). The script includes steps for:

1. Cleaning raw data.
1. Cleaning mapping data.
1. Mapping fields from the raw data based on the cleaned mapping data.
1. Further cleaning of mapped fields.
1. Duplicating each row based on the number of houses it represents.
1. Merging duplicated IDs back to the original data and assigning new IDs.
1. Filtering only the necessary columns for downstream analysis.

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import json
import os

## 2. Define Functions

### 2.1. Clean Raw Data

In [ ]:
# Clean raw data
def clean_raw_data(df):
    
    """Cleans the raw data by padding zeros, replacing values and adding new columns."""

    # Pad leading zeros unless it's '00'
    for column in ['BLD', 'HHT2', 'YBL']:
        df[column] = df[column].astype(str).str.zfill(2).str.replace('00','0')

    # Update GQ records weights
    for column in ['WGTP', 'PWGTP']:
        df[column] = df[column].replace(0, 1)

    # Pad leading zeros for PUMA
    df['PUMA'] = df['PUMA'].astype(str).str.zfill(5)
    
    # Add new columns
    df['num_houses'] = df['WGTP'].astype(int) / df['NP'].astype(int)
    df['num_houses_rounded_up'] = np.ceil(df['num_houses'])

    # Reset index for unique ID
    df.reset_index(inplace=True)

    return df

### 2.2. Clean Mapping Data

In [ ]:
def clean_mapping_data(df):

    """Cleans the mapping data by parsing out item and range fields from values."""

    # Clean up the values field, parse out
    df['values'] = df['values'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    df['item'] = df['values'].apply(lambda x: x.get('item'))
    df['range'] = df['values'].apply(lambda x: x.get('range'))

    return df

### 2.3. Field Mapping

In [ ]:
def field_mapping(df, df_mapping):

    """Using mapping data, apply mapping to the raw data elements"""

    # Filter dataframe containing mappings
    df_allmaps = df_mapping.loc[~df_mapping['item'].isna()]

    # Apply each mapping
    for i in range(len(df_allmaps)):
        row = df_allmaps.iloc[i]
        cur_colname = row['name']
        cur_new_colname = f'map_{cur_colname}'
        cur_map = json.loads(str(row['item']).replace('\'', '\"'))

        # Add mapped values as new column
        df[cur_new_colname] = df[cur_colname].astype(str).replace(cur_map)

    return df

### 2.4. Clean Mapped Fields

In [ ]:
def clean_mapped_fields(df):

    """Creates a list of multiplicated IDs based on the number of houses a record represents."""

    # Year Built cleanup
    df['map_YBL'] = df['map_YBL'].str.replace('N/A \(GQ\)', '', regex=True)
    df['map_YBL'] = df['map_YBL'].str.replace('1939 or earlier', '1900 to 1939', regex=True)
    df[['YBL_range_start','YBL_range_end']] = df['map_YBL'].str.split('(to|or)', expand=True)[[0,2]]
    df = df.replace('', np.nan, regex=True)
    df['YBL_range_start'] = df['YBL_range_start'].astype(float).astype("Int64")
    df['YBL_range_end'] = df['YBL_range_end'].astype(float).astype("Int64")

    # Mapping boundaries for each building type
    map_building_to_unit_boundaries ={
        "One-family house detached":"1",
        "50 or More Apartments":"50 250",
        "Mobile Home or Trailer":"1 250",
        "10-19 Apartments":"10 19",
        "5-9 Apartments":"5 9",
        "20-49 Apartments":"20 49",
        "One-family house attached":"1",
        "3-4 Apartments":"3 4",
        "2 Apartments":"2",
        "N/A (GQ)":"",
        "Boat, RV, van, etc.":"1"
    }

    # Apply mapping
    df['map_BLD2'] = df['map_BLD'].replace(map_building_to_unit_boundaries, regex=False)
    df[['BLD_range_start','BLD_range_end']] = df['map_BLD2'].str.split(' ', expand=True)[[0,1]]
    df[['BLD_range_start','BLD_range_end']] = df[['BLD_range_start','BLD_range_end']].replace('',None)
    df['BLD_range_start'] = df['BLD_range_start'].astype(float).astype("Int64")
    df['BLD_range_end'] = df['BLD_range_end'].astype(float).astype("Int64")

    # Convert WGTP to integer
    df['WGTP'] = df['WGTP'].astype(int)

    return df

### 2.5. Create Multiplicated IDs

In [ ]:
def create_multiplicated_ids(df, multiplicative_surplus_value):

    """Creates a list of multiplicated IDs based on the number of houses a record represents."""
    
    df = df.copy().query('TYPE == 1')
    list_multiplicated_ids = []
    
    for _, row in df.iterrows():
        list_multiplicated_ids.append([row['index']] * int(row['num_houses_rounded_up'] * multiplicative_surplus_value))
    
    list_flattened_multiplicated_ids = [item for sublist in list_multiplicated_ids for item in sublist]
    series_flattened_multiplicated_ids = pd.Series(list_flattened_multiplicated_ids).rename('indexes')
    
    return series_flattened_multiplicated_ids

### 2.6. Join Multiplicated IDs to Data

In [ ]:
def join_multiplicated_ids(df, series):

    """Merges the multiplicated IDs back to the original data and assigns new IDs."""
    
    df = df.merge(series, how = 'outer', left_on = 'index', right_on = 'indexes')
    df = df.reset_index().rename(columns={'level_0':'house_id'})
    return df

### 2.7. Filter Important Columns

In [ ]:
def filter_important_columns(df):

    """Filters only the necessary columns for downstream analysis."""

    list_important_cols = [
        'house_id',
        'PUMA',
        'BDSP',
        'BLD',
        'RMSP',
        'TYPE',
        'YBL',
        'map_BLD',
        'BLD_range_start',
        'BLD_range_end',
        'map_YBL',
        'YBL_range_start',
        'YBL_range_end'
    ]
    return df[list_important_cols]


## 3. Load Data

In [ ]:
df_raw = pd.read_csv(f'../../SupportingDocs/Housing/01_Raw/PUMS_housing_data.csv')
df_mapping = pd.read_csv(f'../../SupportingDocs/Housing/01_Raw/PUMS_housing_variable_mappings.csv')

## 4. Perform Cleaning and Mapping

In [ ]:
df_raw_cleaned = clean_raw_data(df_raw)
df_mapping_cleaned = clean_mapping_data(df_mapping)
df_mapped = field_mapping(df_raw_cleaned, df_mapping_cleaned)
df_mapped_clean = clean_mapped_fields(df_mapped)

## 5. Perform Row Duplication

#### Why Duplicate Rows?

The duplication of rows in our dataset is a critical step in achieving a 1:1 relationship between each record and a house. Here's why:

The PUMS (Public Use Microdata Sample) data we have comes in a form where a single record doesn't necessarily represent a single house. This is because the dataset comes with associated weights and various other fields such as the number of people living in the house.

Let's consider an example to illustrate this:

Imagine a PUMS record describes a two-bedroom apartment with a housing weight of 200. If we assume that the number of people in the house is 4, the number of houses that this record represents is computed as: 
   
   `NumberRepresentedHouses = Housing Weight / Number Of People In House`
   
which in this case equals 50. This means that this one row stands for 50 similar houses (two-bedroom apartments occupied by 4 people).

Therefore, to reach our aim of making one row correspond to one house, we need to duplicate each record by the number of houses it represents.

By doing this, we effectively "flatten" our data. This simplifies our downstream analysis because we can now make direct, house-to-house comparisons using our data. This wouldn't be possible without this duplication step due to the weighted nature of the PUMS data.

As a cautionary note, duplicating rows increases the size of our data. Therefore, it's essential to ensure that your environment has enough memory to handle the increased data size.

In [ ]:
series_multiplicated_ids = create_multiplicated_ids(df_mapped_clean, multiplicative_surplus_value=2)
df_joined = join_multiplicated_ids(df_mapped_clean, series_multiplicated_ids)

## 6. Filter Columns and Prepare for Analysis

In [ ]:
df_filtered = filter_important_columns(df_joined)

## 7. Save

In [ ]:
dir_wrangled = '../../SupportingDocs/Housing/02_Wrangled'

os.makedirs(dir_wrangled, exist_ok=True)

In [ ]:
df_filtered.to_csv(f'{dir_wrangled}/PUMS_housing_1to1.csv',index=False)